Importing Libraries


In [2]:
!pip install opendatasets
import opendatasets as od
import os
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator as IDG
from keras.applications import ResNet50V2, VGG16, Xception, InceptionV3, DenseNet121
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_crossentropy
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential

Getting the Data


In [3]:
dataset = "https://www.kaggle.com/datasets/utkarshsaxenadn/fast-food-classification-dataset"

In [ ]:
od.download(dataset)

In [ ]:
data_dir = "./fast-food-classification-dataset"
os.listdir(data_dir)

In [6]:
train_path = "fast-food-classification-dataset/Fast Food Classification V2/Train"
valid_path = "fast-food-classification-dataset/Fast Food Classification V2/Valid"
test_path = "fast-food-classification-dataset/Fast Food Classification V2/Test"

In [ ]:
class_names = sorted(os.listdir(train_path))
n_classes = len(class_names)
print("No Classes: {}" .format(n_classes))
print("Classes: {}" .format(class_names))

In [8]:
class_dis_train = [len(os.listdir(os.path.join(train_path, name))) for name in class_names]
class_dis_valid = [len(os.listdir(os.path.join(valid_path, name))) for name in class_names]

EDA


In [ ]:
plt.barh(class_names, class_dis_train)
plt.title("Training Class Distribution")
plt.show()

plt.barh(class_names, class_dis_valid)
plt.title("Validation Class Distribution")
plt.show()

class_dis = [len(os.listdir(train_path)) for name in class_names]

fig = px.pie(names = class_names, values = class_dis, hole = 0.3)
fig.update_layout({"title": {'text': "Training class distribution", 'x':0.48}})
fig.show()

fig  = px.bar(x = class_names, y = class_dis, color = class_names)
fig.show()

class_dis = [len(os.listdir(valid_path)) for names in class_names]

fig = px.pie(names = class_names, values = class_dis, hole = 0.3)
fig.update_layout({"title": {'text': "Validation Class Distribution", 'x':0.48}})
fig.show()

fig = px.bar(x = class_names, y = class_dis, color = class_names)
fig.show()

Data Preprocessing

In [ ]:
train_gen = IDG(rescale = 1./255, rotation_range = 10, horizontal_flip = True, vertical_flip = False)
valid_gen = IDG(rescale = 1./255)

train_ds = train_gen.flow_from_directory(train_path, shuffle = True, batch_size = 64, target_size = (256,256), class_mode = 'categorical')
valid_ds = valid_gen.flow_from_directory(valid_path, shuffle = True, batch_size = 32, target_size = (256,256), class_mode = 'categorical')

RESNET


In [ ]:
base_model = ResNet50V2(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.6)(x)
predictions = Dense(len(class_names), activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
for layer in base_model.layers:
  layer.trainable = False

cbs = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint('food-classifier_resnet50v2.h5', save_best_only=True)
]

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4),
    metrics=['accuracy']
)

history = model.fit(train_ds, validation_data=valid_ds, epochs=10, callbacks=cbs)

plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc = 'lower right')
plt.savefig('Resnet_Acc.png')
plt.show()

plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc = 'lower right')
plt.savefig('Resnet_Loss.png')
plt.show()

VGG16


In [ ]:
base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.6)(x)
predictions = Dense(len(class_names), activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
for layer in base_model.layers:
  layer.trainable = False

cbs = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint('food-classifier_resnet50v2.h5', save_best_only=True)
]

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4),
    metrics=['accuracy']
)

history = model.fit(train_ds, validation_data=valid_ds, epochs=10, callbacks=cbs)
plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc = 'lower right')
plt.savefig('VGG16_Acc.png')
plt.show()

plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc = 'lower right')
plt.savefig('VGG16_Loss.png')
plt.show()

Xception

In [ ]:
base_model = Xception(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.6)(x)
predictions = Dense(len(class_names), activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
for layer in base_model.layers:
  layer.trainable = False

cbs = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint('food-classifier_resnet50v2.h5', save_best_only=True)
]

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4),
    metrics=['accuracy']
)

history = model.fit(train_ds, validation_data=valid_ds, epochs=10, callbacks=cbs)


plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc = 'lower right')
plt.savefig('Xception_Acc.png')
plt.show()

plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc = 'lower right')
plt.savefig('Xception_Loss.png')
plt.show()

InceptionV3


In [ ]:
base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.6)(x)
predictions = Dense(len(class_names), activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
for layer in base_model.layers:
  layer.trainable = False

cbs = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint('food-classifier_resnet50v2.h5', save_best_only=True)
]

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4),
    metrics=['accuracy']
)

history = model.fit(train_ds, validation_data=valid_ds, epochs=10, callbacks=cbs)

plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc = 'lower right')
plt.savefig('Inception_Acc.png')
plt.show()

plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc = 'lower right')
plt.savefig('Inception_Loss.png')
plt.show()

DenseNet121

In [ ]:
base_model = DenseNet121(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.6)(x)
predictions = Dense(len(class_names), activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
for layer in base_model.layers:
  layer.trainable = False

cbs = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint('food-classifier_resnet50v2.h5', save_best_only=True)
]

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4),
    metrics=['accuracy']
)

history = model.fit(train_ds, validation_data=valid_ds, epochs=10, callbacks=cbs)

plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc = 'lower right')
plt.savefig('Dense_Net_Acc.png')
plt.show()

plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc = 'lower right')
plt.savefig('Dense_Net_Loss.png')
plt.show()